In [8]:
# image data 불러오는 코드

In [1]:
# encoder image test

###### MATF TEST #######

# Imports
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import torchvision
import numpy as np
import torch.nn.functional as F
from utils import weights_init, conv2DBatchNormRelu, conv2DRelu, deconv2DBatchNormRelu, deconv2DRelu
import torchvision.models as models
import matplotlib.pyplot as plt
import torchvision.transforms as T
import os
from PIL import Image

In [2]:
class ImageDataset(Dataset):
    """
    Creates a PyTorch dataset from folder, returning two tensor images.
    Args: 
    main_dir : directory where images are stored.
    transform (optional) : torchvision transforms to be applied while making dataset
    """

    def __init__(self, main_dir, transform=None):
        self.main_dir = main_dir
        self.transform = transform
        self.all_imgs = os.listdir(main_dir)
        
        img_list=[]
        img_tensor_list=[]
        
        all_files = os.listdir(self.main_dir)
        
        for file in all_files:
            path = main_dir+file
            image = Image.open(path).convert("RGB")
            if self.transform is not None:
                tensor_image = self.transform(image)
            img_list.append(image)
            img_tensor_list.append(tensor_image)

            
        
        self.list_img = img_list
        self.list_ten_img = img_tensor_list

        

            
        
    def __len__(self):
        return len(self.all_imgs)

    def __getitem__(self, index):
        out =  [
                self.list_img,
                self.list_ten_img
        ]

        return out

In [3]:
# path = "C:/Users/NGN/dev/Traffino/TRAFFINO/traffino/datasets/waterloo/img_test/769/"
path = "/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/769/"

In [6]:
transforms = T.Compose([T.ToTensor()]) # Normalize the pixels and convert to tensor.
full_dataset = ImageDataset(
    path, 
    transforms
    ) # Create folder dataset.

In [7]:
print(full_dataset[1][1])


[tensor([[[0.0000, 0.0000, 0.0078,  ..., 0.0078, 0.0078, 0.0039],
         [0.0039, 0.0000, 0.0118,  ..., 0.0000, 0.0000, 0.0000],
         [0.0039, 0.0000, 0.0118,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0314, 0.1922, 0.1294,  ..., 0.2902, 0.1725, 0.0039],
         [0.0000, 0.0039, 0.0000,  ..., 0.2863, 0.1765, 0.0039],
         [0.0235, 0.0118, 0.0000,  ..., 0.2980, 0.1882, 0.0196]],

        [[0.0000, 0.0000, 0.0039,  ..., 0.0078, 0.0078, 0.0039],
         [0.0039, 0.0000, 0.0078,  ..., 0.0039, 0.0039, 0.0078],
         [0.0039, 0.0000, 0.0078,  ..., 0.0157, 0.0235, 0.0275],
         ...,
         [0.0353, 0.1961, 0.1333,  ..., 0.2824, 0.1725, 0.0039],
         [0.0039, 0.0078, 0.0039,  ..., 0.2784, 0.1765, 0.0039],
         [0.0235, 0.0118, 0.0000,  ..., 0.2902, 0.1882, 0.0196]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0039, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0039, 0.0000, 0.0000,  ..., 0.0118, 0.0078, 0

In [9]:
print(full_dataset[2][1])
print(type(full_dataset[2][1]))
print(len(full_dataset[2][1]))
print(len(full_dataset[2][1]))

[tensor([[[0.0000, 0.0000, 0.0078,  ..., 0.0078, 0.0078, 0.0039],
         [0.0039, 0.0000, 0.0118,  ..., 0.0000, 0.0000, 0.0000],
         [0.0039, 0.0000, 0.0118,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0314, 0.1922, 0.1294,  ..., 0.2902, 0.1725, 0.0039],
         [0.0000, 0.0039, 0.0000,  ..., 0.2863, 0.1765, 0.0039],
         [0.0235, 0.0118, 0.0000,  ..., 0.2980, 0.1882, 0.0196]],

        [[0.0000, 0.0000, 0.0039,  ..., 0.0078, 0.0078, 0.0039],
         [0.0039, 0.0000, 0.0078,  ..., 0.0039, 0.0039, 0.0078],
         [0.0039, 0.0000, 0.0078,  ..., 0.0157, 0.0235, 0.0275],
         ...,
         [0.0353, 0.1961, 0.1333,  ..., 0.2824, 0.1725, 0.0039],
         [0.0039, 0.0078, 0.0039,  ..., 0.2784, 0.1765, 0.0039],
         [0.0235, 0.0118, 0.0000,  ..., 0.2902, 0.1882, 0.0196]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0039, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0039, 0.0000, 0.0000,  ..., 0.0118, 0.0078, 0

In [10]:
a=full_dataset[2][1][0]
print(a)
print(type(a))
print(a.shape)


tensor([[[0.0000, 0.0000, 0.0078,  ..., 0.0078, 0.0078, 0.0039],
         [0.0039, 0.0000, 0.0118,  ..., 0.0000, 0.0000, 0.0000],
         [0.0039, 0.0000, 0.0118,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0314, 0.1922, 0.1294,  ..., 0.2902, 0.1725, 0.0039],
         [0.0000, 0.0039, 0.0000,  ..., 0.2863, 0.1765, 0.0039],
         [0.0235, 0.0118, 0.0000,  ..., 0.2980, 0.1882, 0.0196]],

        [[0.0000, 0.0000, 0.0039,  ..., 0.0078, 0.0078, 0.0039],
         [0.0039, 0.0000, 0.0078,  ..., 0.0039, 0.0039, 0.0078],
         [0.0039, 0.0000, 0.0078,  ..., 0.0157, 0.0235, 0.0275],
         ...,
         [0.0353, 0.1961, 0.1333,  ..., 0.2824, 0.1725, 0.0039],
         [0.0039, 0.0078, 0.0039,  ..., 0.2784, 0.1765, 0.0039],
         [0.0235, 0.0118, 0.0000,  ..., 0.2902, 0.1882, 0.0196]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0039, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0039, 0.0000, 0.0000,  ..., 0.0118, 0.0078, 0.

In [4]:
class SemanticImageEncoder(nn.Module):
    '''
    Tianyang:
    Simple Convolutional Encoder for semantic map images
    Input size: flexible
    Output size: 1/2 input
    '''

    def __init__(self, in_channels = 3, out_channels = 32):
        super(SemanticImageEncoder, self).__init__()

        self._encoder = nn.Sequential(
            conv2DBatchNormRelu(in_channels = in_channels, n_filters = 16, \
                k_size = 3,  stride = 1, padding = 1),
            conv2DBatchNormRelu(in_channels = 16, n_filters = 16, \
                k_size = 4,  stride = 1, padding = 2),
            nn.MaxPool2d((2, 2), stride=(2, 2)),

            conv2DBatchNormRelu(in_channels = 16, n_filters = out_channels, \
                k_size = 5,  stride = 1, padding = 2),
            )

    def forward(self, input):
        encoded = self._encoder(input.type(torch.cuda.FloatTensor))
        return encoded

In [6]:
image_channels=3
embed_dim_image=32
image_encoder = SemanticImageEncoder(in_channels = image_channels, out_channels = embed_dim_image)

In [13]:
a_s=a.unsqueeze(0)

print(a_s)
print(type(a_s))
print(a_s.shape)

tensor([[[[0.0000, 0.0000, 0.0078,  ..., 0.0078, 0.0078, 0.0039],
          [0.0039, 0.0000, 0.0118,  ..., 0.0000, 0.0000, 0.0000],
          [0.0039, 0.0000, 0.0118,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0314, 0.1922, 0.1294,  ..., 0.2902, 0.1725, 0.0039],
          [0.0000, 0.0039, 0.0000,  ..., 0.2863, 0.1765, 0.0039],
          [0.0235, 0.0118, 0.0000,  ..., 0.2980, 0.1882, 0.0196]],

         [[0.0000, 0.0000, 0.0039,  ..., 0.0078, 0.0078, 0.0039],
          [0.0039, 0.0000, 0.0078,  ..., 0.0039, 0.0039, 0.0078],
          [0.0039, 0.0000, 0.0078,  ..., 0.0157, 0.0235, 0.0275],
          ...,
          [0.0353, 0.1961, 0.1333,  ..., 0.2824, 0.1725, 0.0039],
          [0.0039, 0.0078, 0.0039,  ..., 0.2784, 0.1765, 0.0039],
          [0.0235, 0.0118, 0.0000,  ..., 0.2902, 0.1882, 0.0196]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0039, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0039, 0.0000, 0.0000,  ..., 0

In [7]:
f_a=image_encoder(a_s)
print(f_a)

NameError: name 'a_s' is not defined

In [15]:
def list2batch(seq):
    # assemble a list of elements to batch, batch_idx: 0th dimension
    stacked = torch.tensor(seq[0]).unsqueeze(0)
    i = 1
    l = len(seq)
    while i < l:
        stacked = torch.cat((stacked, torch.tensor(seq[i]).unsqueeze(0)), 0)
        i += 1
    return stacked

In [8]:
# input_list should be a list, whose each element comes from a scene:
# input_list
# num_scenes

In [ ]:
class MultiAgentSceneTest(nn.Module):
    def __init__(self, image_channels=3, embed_dim_image = 32):
        
        super(MultiAgentSceneTest, self).__init__()
        
        self._semantic_image_encoder = SemanticImageEncoder(in_channels = image_channels, out_channels = embed_dim_image)
        self._resnet = resnetShallow()
 
    def forward(self, config, num_scenes, 
            input_list, 
            resample=0, std=1, use_resnet=0):
        # input_list should be a list, whose each element comes from a scene:   


        input_image_list = []
        for i in range(num_scenes):
            input_image_list.append(input_list[i][2][0])
        
        input_image_batch = self.list2batch(input_image_list)
        
        if use_resnet == 0:
            embed_image_batch = self._semantic_image_encoder(input_image_batch.cuda())
        else:
            embed_image_batch = self._resnet(input_image_batch.cuda())
        
        